# HOMEWORK 2

In [29]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from functools import partial
from multiprocessing import Pool
from github import Github
pd.set_option('display.max_rows', 20)

In [2]:
html = requests.get('https://gist.github.com/paulmillr/2657075').text
soup = BeautifulSoup(html)

In [3]:
soup.find("table").find("tbody").findAll("tr")[0]

<tr> <th scope="row">#1</th> <td><a href="https://github.com/fabpot">fabpot</a> (Fabien Potencier)</td> <td>12947</td> <td>San Francisco</td> <td><a href="https://avatars0.githubusercontent.com/u/47313?s=30&amp;v=4" rel="noopener noreferrer" target="_blank"><img height="30" src="https://avatars0.githubusercontent.com/u/47313?s=30&amp;v=4" style="max-width:100%;" width="30"/></a></td></tr>

In [4]:
rank =[]
user =[]
profile_url = []
contributions = []
location = []

In [5]:
table_content = soup.find("table").find("tbody").findAll("tr")
for i in tqdm(range(len(table_content))):
    elmnt = table_content[i]
    rank.append(elmnt.find("th").text[1:])
    
    row_elmnts = elmnt.findAll("td")
    user.append(row_elmnts[0].text)
    profile_url.append(row_elmnts[0].find('a', href=True).get('href'))
    contributions.append(row_elmnts[1].text) 
    location.append(row_elmnts[2].text)

In [6]:
top_256 = pd.DataFrame(data = {'rank': rank, 'user': user, 'profile_url': profile_url, 'contributions': contributions, 'location': location})

In [7]:
top_256.head()

,rank,user,profile_url,contributions,location
0,1,fabpot (Fabien Potencier),https://github.com/fabpot,12947,San Francisco
1,2,andrew (Andrew Nesbitt),https://github.com/andrew,10188,"Somerset, UK"
2,3,taylorotwell (Taylor Otwell),https://github.com/taylorotwell,8376,"Little Rock, AR"
3,4,egoist (EGOIST),https://github.com/egoist,7688,"Chengdu, China"
4,5,HugoGiraudel (Hugo Giraudel),https://github.com/HugoGiraudel,7658,Berlin


In [9]:
g = Github("332b79a96e9b13c07bbb79d260df9728b6617cc6")

In [ ]:
# if working on windows and jupyter we need to write the function in a .py file

# %load function_hw2.py
def stars_mean(g, user_pseudo):
    repos = g.get_user(user_pseudo).get_repos()
    total_stars = 0
    for repo in repos:
        total_stars +=repo.stargazers_count
    # in case repos do not have any stars
    if repos.totalCount == 0:
        print(user_pseudo)
        result = 0
    else:
        result = total_stars/repos.totalCount
    return result

In [10]:
user_list = top_256.user.apply(lambda x: x.split(" ")[0]).to_list()

In [14]:
import function_hw2
# error -> too much requests for the api
p = Pool(processes=12)
mean_users_star = p.map(partial(workers.stars_mean, g), user_list)

RateLimitExceededException: 403 {'documentation_url': 'https://developer.github.com/v3/#abuse-rate-limits', 'message': 'You have triggered an abuse detection mechanism. Please wait a few minutes before you try again.'}

In [13]:
p = Pool(processes=2)
mean_users_star = p.map(partial(workers.stars_mean, g), user_list)

In [36]:
top_256['stars_mean'] = mean_users_star
top_256.sort_values(['stars_mean'], ascending=False).head(20)

,rank,user,profile_url,contributions,location,stars_mean
225,226,bailicangdu (cangdu),https://github.com/bailicangdu,1500,"Shanghai, China",3250.947368
142,143,vinta (Vinta),https://github.com/vinta,2213,"Taipei, Taiwan",3211.666667
18,19,kamranahmedse (Kamran Ahmed),https://github.com/kamranahmedse,4994,Dubai,2453.403509
117,118,mrdoob (Mr.doob),https://github.com/mrdoob,2555,,2088.593750
153,154,nolimits4web (Vladimir Kharlampidi),https://github.com/nolimits4web,2127,"Rostov-on-Don, Russia",1851.666667
44,45,sebastianbergmann (Sebastian Bergmann),https://github.com/sebastianbergmann,4390,"Siegburg, Germany",1812.625000
230,231,junyanz (Jun-Yan Zhu),https://github.com/junyanz,1479,"Berkeley, CA",1809.866667
195,196,ruanyf (Ruan YiFeng),https://github.com/ruanyf,1755,"Shanghai, China",1506.508197
194,195,IanLunn (Ian Lunn),https://github.com/IanLunn,1757,"Devon, UK",1497.722222
208,209,gorhill (Raymond Hill),https://github.com/gorhill,1646,Canada / Québec,1225.190476


In [37]:
top_256.sort_values(['stars_mean'], ascending=False).tail(20)

,rank,user,profile_url,contributions,location,stars_mean
35,36,rafaelfranca (Rafael França),https://github.com/rafaelfranca,4556,"Ottawa, ON",5.606452
79,80,michalbe (Michał Budzyński),https://github.com/michalbe,3103,"Warsaw, Poland",5.458599
144,145,ashleygwilliams (ashley williams),https://github.com/ashleygwilliams,2193,NYC,4.936047
176,177,laanwj (Wladimir J. van der Laan),https://github.com/laanwj,1912,"Eindhoven, The Netherlands",4.560000
197,198,nfultz (Neal Fultz),https://github.com/nfultz,1731,"Los Angeles, CA",4.179487
1,2,andrew (Andrew Nesbitt),https://github.com/andrew,10188,"Somerset, UK",3.740891
10,11,mikermcneil (Mike McNeil),https://github.com/mikermcneil,5736,"Austin, TX",3.676471
25,26,weierophinney (Matthew Weier O'Phinney),https://github.com/weierophinney,4820,"Sioux Falls, SD, USA",3.411538
38,39,stof (Christophe Coevoet),https://github.com/stof,4521,Paris,3.324427
199,200,suissa (Jean Carlo Nascimento),https://github.com/suissa,1726,Curitiba - PR,3.192133
